## Assignment 8

| Member Name      | Part 1 - Install Libraries | Part 2 - Check Data | Part 3 - Create Map  |
|------------------|----------------------------|---------------------|---------------------|
| Luis Felipe Acosta         | Responsible                 |                     |                     |
| Alejandro Ospina        |                            | Responsible          |                     |
| Raúl Amao         |                            |                     | Responsible          |
| Ruth Chávez         |                            |                     |  Responsible                   |
| Manuel Soto         |                            |                     |   Responsible                  |

### Part 1: Installing and Importing Necessary Libraries

In [1]:
# Part 1: Installing and importing the necessary libraries
#!pip install geopandas rasterio folium

import geopandas as gpd
import rasterio
from rasterio.mask import mask
import folium

### Part 2: Checking the Data

In [2]:
# Load the shapefile of Peru departments
shapefile_path = '../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp'
peru_departments = gpd.read_file(shapefile_path)

In [3]:
# Load the raster file
#raster_path = 'C:/Users/raul3/Downloads/TIF FILES/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif'
# Step 2: Check the CRS of the shapefile
print("Shapefile CRS:", peru_departments.crs)

# Step 3: Open the raster and check its CRS
raster_path = 'C:/Users/raul3/Downloads/TIF FILES/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif'
with rasterio.open(raster_path) as src:
    print("Raster CRS:", src.crs)

# Step 4: If they differ, reproject the shapefile to the raster's CRS
if peru_departments.crs != src.crs:
    peru_departments = peru_departments.to_crs(src.crs)
    print("Reprojected shapefile CRS:", peru_departments.crs)

Shapefile CRS: EPSG:4326
Raster CRS: ESRI:54009
Reprojected shapefile CRS: PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [4]:
# Step 5: Print the bounds of the shapefile
print("Shapefile bounds:", peru_departments.total_bounds)

# Step 6: Print the bounds of the raster
with rasterio.open(raster_path) as src:
    print("Raster bounds:", src.bounds)

Shapefile bounds: [-8.13421654e+06 -2.25408918e+06 -6.72319482e+06 -4.77342975e+03]
Raster bounds: BoundingBox(left=-8041000.0, bottom=-1000000.0, right=-7041000.0, top=0.0)


In [5]:
from shapely.geometry import box

# Create a bounding box from the raster's bounds
with rasterio.open(raster_path) as src:
    raster_bounds = src.bounds
    raster_bbox = box(raster_bounds.left, raster_bounds.bottom, raster_bounds.right, raster_bounds.top)

# Clip the shapefile to the raster bounds
peru_departments_clipped = peru_departments.clip(raster_bbox)

# Now proceed with extracting raster data using peru_departments_clipped

In [6]:
print(peru_departments.columns)

Index(['CCDD', 'NOMBDEP', 'CCPP', 'NOMBPROV', 'CCDI', 'NOMBDIST', 'CAPITAL',
       'UBIGEO', 'IDPROV', 'CODIGO', 'CNT_CCPP', 'DESCRIPCIO', 'DESCARGAR',
       'WHATSAPP', 'CONTACTO', 'geometry'],
      dtype='object')


In [7]:
print(peru_departments.head())

  CCDD    NOMBDEP CCPP    NOMBPROV CCDI      NOMBDIST          CAPITAL  \
0   06  CAJAMARCA   05   CONTUMAZA   04      GUZMANGO         GUZMANGO   
1   06  CAJAMARCA   11  SAN MIGUEL   05      EL PRADO         EL PRADO   
2   06  CAJAMARCA   11  SAN MIGUEL   09        NIEPOS           NIEPOS   
3   06  CAJAMARCA   11  SAN MIGUEL   10  SAN GREGORIO     SAN GREGORIO   
4   06  CAJAMARCA   12   SAN PABLO   03      SAN LUIS  SAN LUIS GRANDE   

   UBIGEO IDPROV  CODIGO  CNT_CCPP    DESCRIPCIO           DESCARGAR  \
0  060504   0605  060504        50      GUZMANGO  www.geogpsperu.com   
1  061105   0611  061105        15      EL PRADO  www.geogpsperu.com   
2  061109   0611  061109        41        NIEPOS  www.geogpsperu.com   
3  061110   0611  061110        48  SAN GREGORIO  www.geogpsperu.com   
4  061203   0612  061203         9      SAN LUIS  www.geogpsperu.com   

    WHATSAPP                  CONTACTO  \
0  931381206  juan.suyo@geogpsperu.com   
1  931381206  juan.suyo@geogpsperu.com

In [8]:
# Check the data types of all columns
print(peru_departments.dtypes)

CCDD            object
NOMBDEP         object
CCPP            object
NOMBPROV        object
CCDI            object
NOMBDIST        object
CAPITAL         object
UBIGEO          object
IDPROV          object
CODIGO          object
CNT_CCPP         int64
DESCRIPCIO      object
DESCARGAR       object
WHATSAPP        object
CONTACTO        object
geometry      geometry
dtype: object


### Part 3: Creating an Interactive Map with GeoJson Layer

In [9]:
# Part 3: Creating an interactive map with department geometries and popups

# Step 1: Initialize a Folium map centered on Peru
peru_map = folium.Map(location=[-9.19, -75.0152], zoom_start=6)

# Step 2: Add GeoJson layer with popups for 'CCDD', 'NOMBDEP', 'NOMBDIST', and 'UBIGEO'
for _, row in peru_departments.iterrows():
    # Create a popup with the relevant information
    popup_text = (f"CCDD: {row['CCDD']}<br>"
                  f"NOMBDEP: {row['NOMBDEP']}<br>"
                  f"NOMBDIST: {row['NOMBDIST']}<br>"
                  f"UBIGEO: {row['UBIGEO']}")

    # Add the geometry and the popup to the map
    folium.GeoJson(
        row['geometry'],  # Use the geometry of the department
        name=row['NOMBDEP'],  # Department name as the layer name
        popup=folium.Popup(popup_text, max_width=300)  # Create a popup with the data
    ).add_to(peru_map)

# Step 3: Add LayerControl to toggle between layers
folium.LayerControl().add_to(peru_map)

# Step 4: Save the map as an HTML file
output_path = 'group_2_ass_8_2024_2.html'
peru_map.save(output_path)

print(f'Map with popups saved as: {output_path}')


Map with popups saved as: group_2_ass_8_2024_2.html
